In [28]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [29]:
raw = pd.read_csv( "../data/interim/lazada_review_stemmed_v2.0_1.csv")

In [30]:
raw['review'] = raw['review'].apply(lambda x: x.split(","))

In [31]:
raw['review'][0]

['kirim',
 'lalu',
 'ninja',
 'sangat',
 'lama',
 'jauh',
 'beda',
 'dengan',
 'kurir',
 'internal',
 'lazada',
 'lebih',
 'baik',
 'kasih',
 'opsi',
 'langgan',
 'agar',
 'bisa',
 'pilih',
 'kurir']

In [32]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [38]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
sentences = raw['review']

Training model...


In [39]:
model = word2vec.Word2Vec(sentences, workers=num_workers,
            size=num_features, min_count = min_word_count,
            window = context, sample = downsampling)
model.init_sims(replace=True)
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-07-10 22:20:56,353 : INFO : collecting all words and their counts
2018-07-10 22:20:56,355 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-10 22:20:56,387 : INFO : PROGRESS: at sentence #10000, processed 95241 words, keeping 2799 word types
2018-07-10 22:20:56,407 : INFO : collected 4668 word types from a corpus of 228695 raw words and 19898 sentences
2018-07-10 22:20:56,408 : INFO : Loading a fresh vocabulary
2018-07-10 22:20:56,412 : INFO : min_count=40 retains 527 unique words (11% of original 4668, drops 4141)
2018-07-10 22:20:56,413 : INFO : min_count=40 leaves 209230 word corpus (91% of original 228695, drops 19465)
2018-07-10 22:20:56,416 : INFO : deleting the raw counts dictionary of 4668 items
2018-07-10 22:20:56,421 : INFO : sample=0.001 downsamples 78 most-common words
2018-07-10 22:20:56,422 : INFO : downsampling leaves estimated 128216 word corpus (61.3% of prior 209230)
2018-07-10 22:20:56,426 : INFO : estimated required memory for 5

In [42]:
model.doesnt_match("telepon genggam pribadi".split())

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2018-07-10 22:22:08,550 : WARNING : vectors for words {'pribadi'} are not present in the model, ignoring these words


'genggam'

In [43]:
model.most_similar("kurir")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('lex', 0.8758602142333984),
 ('express', 0.7913638949394226),
 ('ninja', 0.7905889749526978),
 ('antar', 0.7631889581680298),
 ('rumah', 0.7550110816955566),
 ('one', 0.7262083292007446),
 ('ramah', 0.7044457793235779),
 ('layan', 0.6636135578155518),
 ('bayar', 0.6461251378059387),
 ('tempat', 0.6450743079185486)]

In [44]:
model.most_similar("mantap")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('jiwa', 0.8805218935012817),
 ('pokok', 0.8550896644592285),
 ('keren', 0.8377587795257568),
 ('rekomendasi', 0.8309886455535889),
 ('ekspektasi', 0.7913753986358643),
 ('atas', 0.7903497815132141),
 ('elegan', 0.7794321775436401),
 ('riah', 0.7775986790657043),
 ('oke', 0.7714241147041321),
 ('jangkau', 0.7655647397041321)]

In [45]:
model.most_similar("terima")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('lazada', 0.7417013049125671),
 ('darat', 0.6755008101463318),
 ('selamat', 0.6323566436767578),
 ('gawai', 0.6299276947975159),
 ('tuju', 0.6015115976333618),
 ('kamu', 0.5913076400756836),
 ('kerja', 0.5676023364067078),
 ('selera', 0.5665611624717712),
 ('jaya', 0.5631452798843384),
 ('alhamdulilah', 0.548073410987854)]

In [46]:
model.most_similar("kasih")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('gawai', 0.6962838172912598),
 ('selera', 0.6405074596405029),
 ('sukses', 0.6117714643478394),
 ('jaya', 0.5894997715950012),
 ('makin', 0.5744777917861938),
 ('lazada', 0.560572624206543),
 ('maju', 0.556810736656189),
 ('belanja', 0.5544381141662598),
 ('selalu', 0.5421640276908875),
 ('dan', 0.5401619672775269)]

In [47]:
model.most_similar("jelek")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cewek', 0.9727684259414673),
 ('kaya', 0.9525511860847473),
 ('cowok', 0.9371239542961121),
 ('tali', 0.9338981509208679),
 ('jarum', 0.9326016902923584),
 ('sedih', 0.9300791025161743),
 ('putus', 0.9272193908691406),
 ('belah', 0.918164849281311),
 ('belakang', 0.9142349362373352),
 ('dipake', 0.9119396805763245)]

In [48]:
model.most_similar("kecewa")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('pernah', 0.8381374478340149),
 ('alas', 0.7752129435539246),
 ('customer', 0.764339804649353),
 ('jual', 0.7579182982444763),
 ('tipu', 0.7397640943527222),
 ('tolong', 0.7291659116744995),
 ('sama', 0.7271352410316467),
 ('keluh', 0.7217822074890137),
 ('ambil', 0.7212169766426086),
 ('rusak', 0.7170406579971313)]